In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

## Counting content of Dataset

In [3]:
L = 0
for dirpath, dirnames, filenames in os.walk("../input/accident/ACCIDENT_DATASET"):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'")
    L = L + len(filenames)

## Example Image from Dataset

## Basic Image Processing

In [4]:
# Grayscale
def rgb2gray(rgb_img):
    output_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2GRAY)
    return output_img

In [5]:
# Scale to 0 to 1
def scale(image):
    return image / 255

In [6]:
#resize
def resize_img(image, rows=224, cols=224):
    return cv2.resize(image, dsize=(rows, cols), interpolation=cv2.INTER_CUBIC)

In [7]:
# resize the shape
def reshape(image, axis):
    return np.expand_dims(image.mean(axis=axis), axis=1)

In [8]:
# Function to call other Preprocessing Functions
def preprocessed_img(input_img):
    output_img = rgb2gray(input_img)
    output_img = scale(output_img)
    output_img = resize_img(output_img)
    output_img = reshape(output_img, 1)
    return output_img

## Creating matrices to store converted images and assigning class lebels

In [10]:
# matrix containing L vectors of shape (224, 1)
mean_vector_matrix = np.zeros(shape=(L, 224, 1))
# Target vector containing the classes for L images
target_vector = np.zeros(shape=(L, 1))

In [12]:
n = 0
for root, dirnames, filenames in os.walk(r"../input/accident/ACCIDENT_DATASET/No Accident"):
    n_total = len(filenames)
    for filename in filenames:
        file_path = os.path.join(root, filename)
        img = cv2.imread(file_path)
        img = preprocessed_img(img)
        mean_vector_matrix[n] = img
        target_vector[n] = 1
        if n % 40 == 0:
            print(f"File {n} {filename}")
        n = n + 1  

In [13]:
for root, dirnames, filenames in os.walk(r"../input/accident/ACCIDENT_DATASET/Accident"):
    n_total = len(filenames)
    for filename in filenames:
        file_path = os.path.join(root, filename)
        img = cv2.imread(file_path)
        img = preprocessed_img(img)
        mean_vector_matrix[n] = img
        target_vector[n] = 0
        if n % 40 == 0:
            print(f"File {n} {filename}")
        n = n + 1  

In [14]:

SEED = 0
x_train, x_test, y_train, y_test = train_test_split(mean_vector_matrix, target_vector, test_size=0.2, random_state=SEED)
tf.random.set_seed(SEED)
np.random.seed(SEED)
input_shape = x_train.shape
input_shape


## CNN Model creation

In [15]:
model = tf.keras.Sequential([
   tf.keras.layers.Flatten(),
   tf.keras.layers.Dense(units=200, activation='relu', input_shape = input_shape),
   tf.keras.layers.Dense(units=200, activation='relu', input_shape = input_shape),
   tf.keras.layers.Dense(units=2, activation='softmax')
])

In [16]:
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer = 'sgd',
  metrics = ['accuracy']
)

In [23]:
#try training more times for better results
history = model.fit(
  x = x_train,
  y = y_train,
  epochs = 500
)

## Extra

In [24]:
model.summary()

In [25]:
plt.plot(history.history['accuracy'], label='accuracy')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)

In [26]:
model.save("./model.h5")

In [27]:
from IPython.display import FileLink
FileLink(r'./model.h5')